<a href="https://colab.research.google.com/github/OMWANDHOJR/Alx-Movie-recommendation/blob/main/recommendation_3_4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

alx_movie_recommendation_project_2025_path = kagglehub.competition_download('alx-movie-recommendation-project-2025')

print('Data source import complete.')


100%|██████████| 239M/239M [00:01<00:00, 176MB/s]

Extracting files...


Data source import complete.


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!ls /kaggle/input/alx-movie-recommendation-project-2025/

ls: cannot access '/kaggle/input/alx-movie-recommendation-project-2025/': No such file or directory


In [6]:
!pip install kagglehub
import kagglehub
kagglehub.login()

# Download the dataset
alx_movie_recommendation_project_2025_path = kagglehub.competition_download('alx-movie-recommendation-project-2025')

import os
import pandas as pd

# Assuming the CSV files are directly within the downloaded directory
data_dir = alx_movie_recommendation_project_2025_path

# Update the file paths for reading the CSV files
movies = pd.read_csv(os.path.join(data_dir, 'movies.csv'))
imdb = pd.read_csv(os.path.join(data_dir, 'imdb_data.csv'))
genome_tags = pd.read_csv(os.path.join(data_dir, 'genome_tags.csv'))
genome_scores = pd.read_csv(os.path.join(data_dir, 'genome_scores.csv'))
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
tags = pd.read_csv(os.path.join(data_dir, 'tags.csv'))
links = pd.read_csv(os.path.join(data_dir, 'links.csv'))

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Kaggle credentials set.
Kaggle credentials successfully validated.


In [6]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [7]:
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [8]:
test.isnull().sum()

,0
userId,0
movieId,0


In [ ]:
!pip install kagglehub
import kagglehub
kagglehub.login()

# Download the dataset
alx_movie_recommendation_project_2025_path = kagglehub.competition_download('alx-movie-recommendation-project-2025')

Kaggle credentials set.
Kaggle credentials successfully validated.


In [7]:

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


# Calculate average rating for each movie
movie_ratings = train.groupby('movieId')['rating'].mean()
# Calculate average rating for each user
user_ratings = train.groupby('userId')['rating'].mean()

# Create dictionaries to store averages for faster lookup
movie_avg_ratings = movie_ratings.to_dict()
user_avg_ratings = user_ratings.to_dict()

# Calculate overall average rating
overall_avg_rating = train['rating'].mean()

# Function to predict ratings for the test data
def predict_rating(row):
    movie_id = row['movieId']
    user_id = row['userId']

    # Get movie and user average ratings
    movie_avg = movie_avg_ratings.get(movie_id, overall_avg_rating)
    user_avg = user_avg_ratings.get(user_id, overall_avg_rating)

    # Predict rating using user and movie biases
    predicted_rating = overall_avg_rating + (user_avg - overall_avg_rating) + (movie_avg - overall_avg_rating)

    return predicted_rating
# 1. Entity Featurization using TF-IDF
# Combine relevant text features for each movie
movies['features'] = movies['genres'].fillna('')  # Start with genres

# Create a TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the movie features
tfidf_matrix = tfidf.fit_transform(movies['features'])

# 2. Similarity Computation using Cosine Similarity
# Calculate cosine similarity between movies based on their TF-IDF features
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. Recommendation Function using Similarity
def recommend_movies_similarity(user_id, top_n=10):
    # Get the movies the user has rated
    user_movies = train[train['userId'] == user_id]['movieId'].tolist()

    # Get the indices of these movies in the movies DataFrame
    movie_indices = movies[movies['movieId'].isin(user_movies)].index

    # Calculate the average similarity scores for the user's movies
    similarity_scores = cosine_sim[movie_indices].mean(axis=0)

    # Get the indices of movies sorted by similarity (excluding user's movies)
    sorted_indices = np.argsort(similarity_scores)[::-1]
    recommended_indices = [i for i in sorted_indices if i not in movie_indices][:top_n]

    # Get the movie IDs of the recommended movies
    recommended_movies = movies.iloc[recommended_indices]['movieId'].tolist()

    return recommended_movies

# Example usage: Recommend top 10 movies for user ID 1
recommended_movies = recommend_movies_similarity(1)
print(f"Recommended movies for user ID 1 (using similarity): {recommended_movies}")



Recommended movies for user ID 1 (using similarity): [90, 209139, 124, 127, 14, 114, 209145, 209157, 121, 55]


In [9]:
# Create the 'Id' column in the test DataFrame
test['Id'] = test['userId'].astype(str) + '_' + test['movieId'].astype(str)

# Apply the predict_rating function to create the 'predicted_rating' column
test['predicted_rating'] = test.apply(predict_rating, axis=1) # Apply the predict_rating function to each row

# Select the required columns and save to CSV
submission_df = test[['Id', 'predicted_rating']]
submission_df.columns = ['Id', 'rating']  # Rename 'predicted_rating' to 'rating'
submission_df.to_csv('/kaggle/submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
